In [ ]:
import os
import tensorflow as tf
import IPython
import numpy as np
import pandas as pd
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator
# Deep learning libraries
import keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# Setting seeds for reproducibility
seed = 69
np.random.seed(seed)
tf.random.set_seed(seed)


In [ ]:
def change_contrast(img):
    return tf.image.adjust_contrast(img, 1.5)

In [ ]:
input_path = '/content/drive/My Drive/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
for st in ['train', 'val', 'test']:
    n_03 = len(os.listdir(input_path + '/' + st + '/03'))
    n_48 = len(os.listdir(input_path + '/' + st + '/48'))
    n_913 = len(os.listdir(input_path + '/' + st + '/913'))
    n_1418 = len(os.listdir(input_path + '/' + st + '/1418'))
    print('Set: {}, 03 images: {}, 48 images: {}, 913 images: {}, 1418 images: {}'.format(st, n_03, n_48, n_913, n_1418))

Set: train, 03 images: 612, 48 images: 1323, 913 images: 1343, 1418 images: 476
Set: val, 03 images: 39, 48 images: 83, 913 images: 85, 1418 images: 31
Set: test, 03 images: 115, 48 images: 248, 913 images: 251, 1418 images: 89


In [ ]:
def process_data(img_dims, batch_size):
    train_datagen = ImageDataGenerator(rotation_range=5, 
                                       rescale=1./255, 
                                       zoom_range=0.07, 
                                       width_shift_range=0.05,
                                       height_shift_range=0.05,
                                       shear_range=0.07,
                                       preprocessing_function=change_contrast)
    
    test_val_datagen = ImageDataGenerator(rescale=1./255,
                                          preprocessing_function=change_contrast)
    
    train_gen = train_datagen.flow_from_directory(directory=input_path+ '/' + 'train', 
                                                  target_size=(img_dims, img_dims), 
                                                  batch_size=batch_size, 
                                                  class_mode='categorical', 
                                                  shuffle=True)

    test_gen = test_val_datagen.flow_from_directory(directory=input_path+ '/' + 'test', 
                                                    target_size=(img_dims, img_dims), 
                                                    batch_size=batch_size, 
                                                    class_mode='categorical', 
                                                    shuffle=False)
    
    test_data = []
    test_labels = []

    for cond in ['/03/', '/48/', '/913/', '/1418/']:
        for img in (os.listdir(input_path + '/' + 'test' + cond)):
            img = cv2.imread(input_path+ '/' + 'test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = change_contrast(img)
            if cond=='/48/':
                label = 1
            elif cond=='/913/':
                label = 2
            elif cond=='/03/':
                label = 0
            elif cond=='/1418/':
                label = 3
            test_data.append(img)
            test_labels.append(label)
    test_data = np.array(test_data) / 255.0
    test_labels = np.array(test_labels)
    
    return train_gen, test_gen, test_data, test_labels

In [ ]:
img_dims = 224
batch_size = 64

# Getting the data
train_gen, test_gen, test_data, test_labels = process_data(img_dims, batch_size)

Found 3754 images belonging to 4 classes.
Found 703 images belonging to 4 classes.


In [ ]:
!git clone https://github.com/WittmannF/LRFinder.git
from LRFinder.keras_callback import LRFinder

Cloning into 'LRFinder'...
remote: Enumerating objects: 71, done.
remote: Total 71 (delta 0), reused 0 (delta 0), pack-reused 71
Unpacking objects: 100% (71/71), done.


In [ ]:
lr_finder = LRFinder(min_lr=1e-4, max_lr=1)

In [ ]:
pip install -U keras-efficientnet-v2

In [ ]:
import keras_efficientnet_v2
model = keras_efficientnet_v2.EfficientNetV2L(pretrained="imagenet", num_classes=4, include_preprocessing=False)

In [ ]:
 base_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_dims, img_dims, 3))
#base_model = tf.keras.applications.DenseNet201(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_dims, img_dims, 3), pooling=None)
base_model.trainable = True
head_model = base_model.output
head_model = Flatten()(head_model)
head_model = Dense(512, activation='relu')(head_model)
head_model = Dropout(0.3)(head_model)
head_model = BatchNormalization()(head_model)
head_model = Dense(256, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = BatchNormalization()(head_model)
predictions = Dense(4, activation='softmax')(head_model)
model = tf.keras.Model(base_model.input, outputs=predictions)

opt = Adam(learning_rate=1e-3)
class_weight = {0: 2.194,
                1: 1.015,
                2: 1,
                3: 2.821}
es = tf.keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.05, patience=6)
lr_reduce = ReduceLROnPlateau(monitor='loss', factor=0.3, patience=3, mode='auto')
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model = tf.keras.applications.MobileNetV3Small(
    input_shape=(img_dims, img_dims, 3), alpha=1.0, minimalistic=False, include_top=False,
    weights='imagenet', input_tensor=None, classes=4, pooling=None,
    dropout_rate=0.2, classifier_activation='softmax',
    include_preprocessing=False
)

4334752/4334752 [==============================] - 0s 0us/step


In [ ]:
model = tf.keras.applications.regnet.RegNetY032(
    model_name='regnety320', include_top=True, include_preprocessing=False,
    weights=None , input_tensor=None, input_shape=None, pooling=None,
    classes=4, classifier_activation='softmax'
)

In [ ]:
model.summary()

In [ ]:
i = 0
for layer in model.layers:
  print(i, layer.name, layer.trainable)
  i = i + 1

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=False, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96,
    layer_range=None, show_layer_activations=False
)

In [ ]:
model.trainable = True
for layer in model.layers:
  layer.trainable = True

In [ ]:
opt = Adam(learning_rate=1e-2)
class_weight = {0: 2.194,
                1: 1.015,
                2: 1.0,
                3: 2.821}
es = tf.keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.05, patience=4)
lr_reduce = ReduceLROnPlateau(monitor='loss', factor=0.3, patience=3, mode='auto')
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
            train_gen, steps_per_epoch=train_gen.samples // batch_size, 
            epochs=5, callbacks=[lr_finder],
            class_weight=class_weight)

In [ ]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)
opt = tf.keras.optimizers.Adam(lr=1e-4)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=4, verbose=2, mode='auto')

In [ ]:
history = model.fit(
            train_gen, steps_per_epoch=train_gen.samples // batch_size, 
            epochs=5, validation_data=test_gen, 
            validation_steps=test_gen.samples // batch_size, callbacks=[lr_reduce])

In [ ]:
tf.keras.models.save_model(model, filepath = '/content/drive/My Drive/dpdpdp')

INFO:tensorflow:Assets written to: /content/drive/My Drive/dpdpdp/assets


In [ ]:
model = keras.models.load_model('/content/drive/My Drive/dpdpdp')

In [ ]:
model.trainable = True

In [ ]:
for layer in model.layers:
  print(layer.trainable)

In [ ]:
lr_finder = LRFinder(min_lr=1e-5, max_lr=1)

In [ ]:
history = model.fit(
            train_gen, steps_per_epoch=train_gen.samples // batch_size, 
            epochs=5, callbacks=[lr_finder],
            class_weight=class_weight)

In [ ]:
opt = Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
lr_reduce = ReduceLROnPlateau(monitor='loss', factor=0.4, patience=4, verbose=2, mode='auto')

In [ ]:
i = 0
for layer in model.layers:
  print(i, layer.name, layer.trainable)
  i = i + 1

In [ ]:
class_weight = {0: 2.194,
                1: 1.015,
                2: 1,
                3: 2.821}

In [ ]:
history = model.fit(
            train_gen, steps_per_epoch=train_gen.samples // batch_size,
            epochs=50, validation_data=test_gen,
            validation_steps=test_gen.samples // batch_size, callbacks=[lr_reduce],
            class_weight=class_weight)

In [ ]:
model.save("/content/drive/My Drive/dddd")

INFO:tensorflow:Assets written to: /content/drive/My Drive/DiplomPredict/assets


In [ ]:
lr_finder = LRFinder(min_lr=1e-4, max_lr=1)

In [ ]:
history = model.fit(
            train_gen, steps_per_epoch=train_gen.samples // batch_size, 
            epochs=5, callbacks=[lr_finder],
            class_weight=class_weight)

In [ ]:
predIdxs = model.predict(test_data)
predIdxs = np.argmax(predIdxs,axis=1)

In [ ]:
cm=confusion_matrix(test_labels, predIdxs, labels =[0, 1, 2, 3])
print(cm.shape)
print(cm)

In [ ]:
tp25 = cm[0][0]
tp50 = cm[1][1]
tp75 = cm[2][2]
tp100 = cm[3][3]

fn25 = cm[0][1] + cm[0][2] + cm[0][3]
fn50 = cm[1][0] + cm[1][2] + cm[1][3]
fn75 = cm[2][0] + cm[2][1] + cm[2][3]
fn100 = cm[3][0] + cm[3][1] + cm[3][2]

fp25 = cm[1][0] + cm[2][0] + cm[3][0]
fp50 = cm[0][1] + cm[2][1] + cm[3][1]
fp75 = cm[0][2] + cm[1][2] + cm[3][2]
fp100 = cm[0][3] + cm[1][3] + cm[2][3]

precision_25 = tp25/(tp25+fp25)*100
precision_50 = tp50/(tp50+fp50)*100
precision_75 = tp75/(tp75+fp75)*100
precision_100 = tp100/(tp100+fp100)*100

recall_25 = tp25/(tp25+fn25)*100
recall_50 = tp50/(tp50+fn50)*100
recall_75 = tp75/(tp75+fn75)*100
recall_100 = tp100/(tp100+fn100)*100

print('Precision 25: {:.2f}%'.format(precision_25))
print('Recall 25: {:.2f}%'.format(recall_25))
print()

print('Precision 50: {:.2f}%'.format(precision_50))
print('Recall 50: {:.2f}%'.format(recall_50))
print()

print('Precision 75: {:.2f}%'.format(precision_75))
print('Recall 75: {:.2f}%'.format(recall_75))
print()

print('Precision 100: {:.2f}%'.format(precision_100))
print('Recall 100: {:.2f}%'.format(recall_100))